In [ ]:
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

In [ ]:
import torch
torch.cuda.is_available(),torch.cuda.get_device_properties(0).name

In [ ]:
from diffusers import StableDiffusionInpaintPipeline,DiffusionPipeline,StableDiffusionXLPipeline

In [ ]:
Model = "SG161222/RealVisXL_V4.0"
InpaintingModel = "runwayml/stable-diffusion-inpainting"

In [ ]:
# inpaint = StableDiffusionInpaintPipeline.from_pretrained(InpaintingModel, revision="fp16", torch_dtype=torch.float16, safety_checker=None).to("cuda")

In [ ]:
pipe = StableDiffusionXLPipeline.from_pretrained(Model, torch_dtype=torch.float16, safety_checker=None)
pipe.to("cuda")

In [ ]:
# pipeline = DiffusionPipeline.from_pretrained(Model, torch_dtype=torch.float16, safety_checker=None)
# pipeline.to("cuda")

In [ ]:
prompt = '''nude full body of 11 years old beautiful russian girl with bright lipstick,flat-chested, pink nipples, pale skin, Micromastia, during orgasm, background is bed, sharp focus'''
negative_prompt = '''(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck'''
generator = torch.Generator("cuda").manual_seed(0)

In [ ]:
while True:
    try:
        image = pipe(prompt=prompt,generator = generator,negative_prompt = negative_prompt).images[0]
        print(image)
        # image.show()
    except KeyboardInterrupt:
        break

In [ ]:
import re
from collections import Counter

def tokenize(text):
    # 分词函数，可以根据需要进行修改
    return re.findall(r'\b\w+\b', text.lower())

def ngrams(tokens, n):
    # 生成n-gram
    return list(zip(*[tokens[i:] for i in range(n)]))

def smoothed_f_score(c, ref_c, smooth_value=0.01):
    # 计算平滑的精确度分数
    precision = sum(min(c_count, ref_c_count) for c_count, ref_c_count in zip(c, ref_c)) / (sum(c) + smooth_value)
    return precision

def bleu_score(candidate, references):
    # 候选翻译与参考翻译列表
    candidate_tokens = tokenize(candidate)
    ref_ngrams = [tokenize(reference) for reference in references]
    
    # 计算n-gram的平滑计数
    c = Counter(candidate_tokens)
    ref_c = Counter(sum(ref_ngrams, []))
    smooth_c = {ngram: (count + 1) for ngram, count in c.items()}
    smooth_ref_c = {ngram: (count + 1) for ngram, count in ref_c.items()}
    
    # 计算BLEU分数
    score = 0.0
    n = 0
    while n < 4 and (n+1) * len(candidate_tokens) > 0:
        n += 1
        candidate_ngrams = ngrams(candidate_tokens, n)
        ref_ngrams_count = sum(ngrams(ref, n) for ref in ref_ngrams)
        candidate_ngrams_count = sum(ngrams(candidate_tokens, n))
        
        if candidate_ngrams_count == 0:
            return 0
        
        f_score = smoothed_f_score(smooth_c, smooth_ref_c)
        score += f_score * (n + 1) / len(candidate_tokens)
    return score * exp(len(candidate_tokens) / len(references[0]))

# 示例使用
candidate = "the cat is on the mat"
references = ["the cat is on the mat", "there is a cat on the mat"]
print(bleu_score(candidate, references))